OPÇÃO PARA GERAR EMBEDDINGS COM 0 OLLAMA

In [2]:
!pip install llama-index-vector-stores-mongodb
!pip install llama-index
!pip install pymongo
!pip install torch
!pip install psutil
!pip install ollama



  Using cached llama_index_vector_stores_mongodb-0.1.5-py3-none-any.whl.metadata (6.5 kB)
  Using cached llama_index_core-0.10.43.post1-py3-none-any.whl.metadata (2.4 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached llamaindex_py_client-0.1.19-py3-none-any.whl.metadata (760 bytes)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached llama_index_vector_stores_mongodb-0.1.5-py3-none-any.whl (7.0 kB)
Using cached llama_index_core-0.10.43.post1-py3-none-any.whl (15.4 MB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
Using cached dirtyjson-1.0.8-py3-none-any.whl (25 kB)
Using cached llamaindex_py_client-0.1.19-py3-none-any.whl (141 kB)
Using cac

Configurando conexão MongoDB

In [3]:
import os
import pymongo
from pymongo import MongoClient
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import ollama
import psutil
username = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')

# Configurar URI do MongoDB
client = MongoClient(f"mongodb+srv://{username}:{password}@{host}")
db = client["SV-Collections"]
games_db = db['Games']
embeddings_db = db['Embeddings-Ollama']
print("ok")

# Configurar o armazenamento de vetores
store = MongoDBAtlasVectorSearch(client)
storage_context = StorageContext.from_defaults(vector_store=store)

# Função para verificar o uso de memória
def check_memory():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Uso de memória: {mem_info.rss / (1024 ** 2)} MB")


ok


In [ ]:
total_games = games_db.count_documents({})
print(total_games)

arregar Modelo de Linguagem

In [4]:
# Puxar o modelo do Ollama
response = ollama.pull('llama3')
print(response)


{'status': 'success'}


Carregar dados e criar Embeddings

In [15]:
# Função para gerar embeddings
def generate_embeddings(prompt, model_name):
    response = ollama.embeddings(model=model_name, prompt=prompt)
    print("Resposta da função ollama.embeddings:", response)
    
    # Verificar se a chave 'embeddings' está presente na resposta
    if 'embedding' in response:
        embeddings = response['embedding']
        return embeddings
    else:
        raise KeyError("A chave 'embedding' não está presente na resposta.")


# Função para buscar dados dos jogos do MongoDB e atualizar embeddings
def update_game_embeddings(games_db, embeddings_db, batch_size=50):
    cursor = games_db.find(batch_size=batch_size)
    processed_games = 0
    
    while cursor.alive:
        batch = cursor.next()
        
        game_id = batch['_id']
        game_title = batch['title']
        # Verificar se o embedding já existe
        if embeddings_db.find_one({'title': game_title}):
            print(f"Embedding já existe para o jogo: {batch['title']}, pulando.")
            continue

        title = batch.get('title', '')
        description = batch.get('description', '')
        developers = batch.get('developers', '')
        genres = ', '.join(batch.get('genres', []))
        plataforms = batch.get('plataforms', [])
        platforms = batch.get('platforms', [])
        combined_platforms = ', '.join(set(plataforms + platforms))
        rating = str(batch.get('rating', ''))
        price = str(batch.get('price', ''))
        tags = ', '.join(batch.get('tags', []))
        release_date = str(batch.get('release_date', ''))

        # Combine todos os campos em um texto único para gerar o embedding
        combined_text = f"Title: {title}\nDescription: {description}\nDevelopers: {developers}\nGenres: {genres}\nPlatforms: {combined_platforms}\nRating: {rating}\nPrice: {price}\nTags: {tags}\nRelease Date: {release_date}"
        
        print(f"Gerando embedding para o jogo: {title}")
        embedding = generate_embeddings(combined_text, 'llama3')

        # Preparar os dados para salvar na coleção embeddings
        embedding_data = {
            'game_id': game_id,
            'title': title,
            'description': description,
            'embedding': embedding,
            'developers': developers,
            'genres': genres,
            'platforms': combined_platforms,
            'rating': rating,
            'price': price,
            'tags': tags,
            'release_date': release_date,
            'combined_text': combined_text
        }
        
        # Salvar embedding no MongoDB
        embeddings_db.update_one(
            {'game_id': game_id},
            {'$set': embedding_data},
            upsert=True
        )
        print(f"Embedding salvo com sucesso para o jogo: {title}")
        
        processed_games += 1
        if processed_games % batch_size == 0:
            print(f"{processed_games} jogos processados até agora.")

# Atualizar os embeddings e armazenar na coleção de embeddings

update_game_embeddings(games_db, embeddings_db)


Embedding já existe para o jogo: Elden Ring, pulando.
Embedding já existe para o jogo: Grand Theft Auto V, pulando.
Embedding já existe para o jogo: Assassin's Creed IV: Black Flag, pulando.
Embedding já existe para o jogo: Diablo III, pulando.
Embedding já existe para o jogo: FIFA 14, pulando.
Embedding já existe para o jogo: Battlefield 4, pulando.
Embedding já existe para o jogo: Just Dance 2014, pulando.
Embedding já existe para o jogo: Minecraft: Xbox 360 Edition, pulando.
Embedding já existe para o jogo: The Last of Us, pulando.
Embedding já existe para o jogo: Tomb Raider, pulando.
Embedding já existe para o jogo: Batman: Arkham Origins, pulando.
Embedding já existe para o jogo: NBA 2K14, pulando.
Embedding já existe para o jogo: Animal Crossing: New Leaf, pulando.
Embedding já existe para o jogo: Call of Duty: Ghosts, pulando.
Embedding já existe para o jogo: Pokémon X and Y, pulando.
Embedding já existe para o jogo: Bioshock Infinite, pulando.
Embedding já existe para o jogo: 

CursorNotFound: cursor id 1915248918652422168 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 1915248918652422168 not found', 'code': 43, 'codeName': 'CursorNotFound', '$clusterTime': {'clusterTime': Timestamp(1717959805, 2), 'signature': {'hash': b'\xaac\xfbZ\xdb\xaa\xa4!\x93\xdc\xc0\x862\xc6\xc3\xf9\xd03\xf0\x18', 'keyId': 7332966662833963009}}, 'operationTime': Timestamp(1717959805, 2)}

0Verificar Embeddings Salvos

In [ ]:
# Verificar os embeddings salvos
def verificar_embeddings_salvos():
    count = store._collection.count_documents({})
    print(f"Número de embeddings salvos: {count}")

verificar_embeddings_salvos()
